In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
import random
import urllib
import csv
from collections import defaultdict
import numpy as np
import pandas as pd
import pickle

In [11]:
def run(network, time, demographic, DF, IF, RF, DM, IM, RM, zealots, mew):
    lessStrict, leftAs, moreStrict, nodeOpinions = [],[],[],[] #Store opinion proportions
    list_DF, list_IF, list_RF, list_DM, list_IM, list_RM = [],[],[],[],[],[] #Store average opinions 
    c1, c2, c3 = 0, 0, 0 #Count final opinion majority
    i = 0
    setup(network, demographic, DF, IF, RF, DM, IM, RM, zealots, mew) 
    nx.write_gpickle(network, "Setup")
    while i < iterations:
        df, If, rf, dm, im, rm = 829, 424, 767, 767, 424, 828
        G = nx.Graph()
        G = nx.read_gpickle("Setup")
        list1, list2, list3, time_list, winner, nodes_av, nodes, DF_c, IF_c, RF_c, DM_c, IM_c, RM_c = update(G, time, mew, df, If, rf, dm, im, rm)
        #Count total final opinion majorities for all iterations
        if winner == 1:
            c1 += 1
        elif winner == 2:
            c2 += 1
        else:
            c3 +=1
        #Add iteration list to complete list:
        lessStrict.append(list1)
        leftAs.append(list2)
        moreStrict.append(list3)
        nodeOpinions.append(nodes_av)
        list_DF.append(DF_c)
        list_IF.append(IF_c)
        list_RF.append(RF_c)
        list_DM.append(DM_c)
        list_IM.append(IM_c)
        list_RM.append(RM_c)
        print(i)
        i += 1
    #Grahs generated data:
    print("lessStrict, leftAs, moreStrict:", c1, c2, c3)
    final1 = [sum(x)/iterations for x in zip(*lessStrict)]
    final2 = [sum(x)/iterations for x in zip(*leftAs)]
    final3 = [sum(x)/iterations for x in zip(*moreStrict)]
    final4 = [sum(x)/iterations for x in zip(*nodeOpinions)]
    final5 = [sum(x)/iterations for x in zip(*list_DF)]
    final6 = [sum(x)/iterations for x in zip(*list_IF)]
    final7 = [sum(x)/iterations for x in zip(*list_RF)]
    final8 = [sum(x)/iterations for x in zip(*list_DM)]
    final9 = [sum(x)/iterations for x in zip(*list_IM)]
    final10 = [sum(x)/iterations for x in zip(*list_RM)]
    
    plt.figure(figsize = [13, 6])
    plt.axhline(y = 0.5, color = 'k', linestyle = '--', label = "Simple Majority Threshold")
    plt.plot(time_list, final1, label = "Less Strict")
    plt.plot(time_list, final2, label = "Left As They Are")
    plt.plot(time_list, final3, label = "More Strict")
    
    plt.ylabel("Count", fontsize = 16)
    plt.xlabel("Time", fontsize = 16)
    plt.xlim(0, time)
    plt.xticks(fontsize = 16)
    plt.ylim(0, 1)
    plt.yticks(fontsize = 16)
    plt.legend(loc = 'upper right')
    plt.show()


    plt.figure(figsize = [13, 6])
    plt.axhline(y = 0.33, color = 'k', linestyle = '--')
    plt.axhline(y = 0.66, color = 'k', linestyle = '--')
    plt.plot(time_list, final4, label = "All")
    plt.plot(time_list, final5, label = "DF")
    plt.plot(time_list, final6, label = "IF")
    plt.plot(time_list, final7, label = "RF")
    plt.plot(time_list, final8, label = "DM")
    plt.plot(time_list, final9, label = "IM")
    plt.plot(time_list, final10, label = "RM")
    plt.ylabel("Avg. Opinion",fontsize = 16)
    plt.xlabel("Time", fontsize = 16)
    plt.xlim(0, time)
    plt.xticks(fontsize = 16)
    plt.ylim(0, 1)
    plt.yticks(fontsize = 16)
    plt.legend(loc = 'upper right')
    plt.show()
    
    FINAL = time_list, final1, final2, final3, final4, final5, final6, final7, final8, final9, final10
    return FINAL

In [12]:
#Homophily Mechanism:
def epsilon(gender0, gender1, party0, party1):
    if (gender0 == gender1) and (party0 == party1):
        e = 0.375
    if (gender0 != gender1) and (party0 == party1):
        e = 0.25
    if (gender0 == gender1) and (party0 != party1):
        e = 0.25
    if (gender0 != gender1) and (party0 != party1):
        e = 0.125
    return e

#Periodic Boundary Conditions:
def rho(x1, x2):
    x = x1 - x2
    p = 0.0
    if x >= -1 and x < -0.5:
        p = -1.0
    elif x >= -0.5 and  x < 0.5:
        p = 0.0
    elif x >= 0.5 and x <= 1:
        p = 1.0
    return p

#Opinion Distance:
def tau(x_i,x_j):
    return abs(x_i - x_j - rho(x_i, x_j))

In [13]:
#Model Constructor
def setup(network, demographic, DF, IF, RF, DM, IM, RM, zealots, mew):
    counter = 0 
    allt = DF, IF, RF, DM, IM, RM 
    cum_allt = np.cumsum(allt) 

    demo_nr = [round(i*len(network.nodes())) for i in dem_per]
    cum_demo_nr = np.cumsum(demo_nr)
    nodes_list=list(network.nodes())
    random.shuffle(nodes_list) #Randomizes node assignment order
    cn = 0
    #Assigns each node demographic characteristics:
    for node in nodes_list:
        if cn < cum_demo_nr[0]:
            network.nodes[node]['gender'] = "Female"
            network.nodes[node]['party'] = "Democrat"
            if cn < cum_allt[0]:
                network.nodes[node]['opinion'] = 0.0
            if cn>=cum_allt[0] and cn < cum_allt[1]:
                network.nodes[node]['opinion'] = 0.5
            if cn>=cum_allt[1] and cn < cum_allt[2]:
                network.nodes[node]['opinion'] = 1.0
        if cn >= cum_demo_nr[0] and cn < cum_demo_nr[1]:
            network.nodes[node]['opinion'] = 0.0
            network.nodes[node]['gender'] = "Female"
            network.nodes[node]['party'] = "Independent"
            if cn>=cum_allt[2] and cn < cum_allt[3]:
                network.nodes[node]['opinion'] = 0.0
            if cn>=cum_allt[3] and cn < cum_allt[4]:
                network.nodes[node]['opinion'] = 0.5
            if cn>= cum_allt[4] and cn < cum_allt[5]:
                network.nodes[node]['opinion'] = 1.0
        if cn >= cum_demo_nr[1] and cn < cum_demo_nr[2]:
            network.nodes[node]['opinion'] = 0.0
            network.nodes[node]['gender'] = "Female"
            network.nodes[node]['party'] = "Republican"
            if cn>=cum_allt[5] and cn < cum_allt[6]:
                network.nodes[node]['opinion'] = 0.0
            if cn>=cum_allt[6] and cn < cum_allt[7]:
                network.nodes[node]['opinion'] = 0.5
            if cn>= cum_allt[7] and cn < cum_allt[8]:
                network.nodes[node]['opinion'] = 1.0
        if cn>= cum_demo_nr[2] and cn < cum_demo_nr[3]:
            network.nodes[node]['opinion'] = 0.0
            network.nodes[node]['gender'] = "Male"
            network.nodes[node]['party'] = "Democrat"
            if cn>= cum_allt[8] and cn < cum_allt[9]:
                network.nodes[node]['opinion'] = 0.0
            if cn>= cum_allt[9] and cn < cum_allt[10]:
                network.nodes[node]['opinion'] = 0.5
            if cn>= cum_allt[10] and cn < cum_allt[11]:
                network.nodes[node]['opinion'] = 1.0
        if cn >= cum_demo_nr[3] and cn < cum_demo_nr[4]:
            network.nodes[node]['opinion'] = 0.0
            network.nodes[node]['gender'] = "Male"
            network.nodes[node]['party'] = "Independent"
            if cn>=cum_allt[11] and cn < cum_allt[12]:
                network.nodes[node]['opinion'] = 0.0
            if cn>=cum_allt[12] and cn < cum_allt[13]:
                network.nodes[node]['opinion'] = 0.5
            if cn>= cum_allt[13] and cn < cum_allt[14]:
                network.nodes[node]['opinion'] = 1.0
        if cn>= cum_demo_nr[4] and cn < cum_demo_nr[5]:
            network.nodes[node]['opinion'] = 0.0
            network.nodes[node]['gender'] = "Male"
            network.nodes[node]['party'] = "Republican"
            if cn>= cum_allt[14] and cn < cum_allt[15]:
                network.nodes[node]['opinion'] = 0.0
            if cn>= cum_allt[15] and cn < cum_allt[16]:
                network.nodes[node]['opinion'] = 0.5
            if cn>= cum_allt[16] and cn < cum_allt[17]:
                network.nodes[node]['opinion'] = 1.0
        network.nodes[node]['zealot'] = "No"
        cn+=1
        
        #Selects the first z randomly selected nodes as zealots: 
        if counter < zealots:
            if network.nodes[node]['opinion'] == 0.0:
                network.nodes[node]['zealot'] = "Yes"
                counter += 1
            else:
                network.nodes[node]['zealot'] = "No"
        elif counter > zealots:
            network.nodes[node]['zealot'] = "No"

    mu = mew #Sets level of social influence to parameter
    return network.nodes 

In [14]:
#Update
def update(network,total_time, mew, df, If, rf, dm, im, rm):
    list1, list2, list3 = [],[],[] #Stores opinion proportions within each iteration
    counter1, counter2, counter3 = (758/4039), (1460/4039), (1813/4039) 
    avg_DF, avg_IF, avg_RF, avg_DM, avg_IM, avg_RM = 0, 0, 0, 0, 0, 0 #Stores average opinions
    DF_l, IF_l, RF_l, DM_l, IM_l, RM_l = [], [], [], [], [], [] #Stores average opionions within each iteration
    time_list = []
    nodes = []
    nodes_av = []
    time = 0
    edges_list = list(network.edges())

    avg_DF = (158*0.5 + 663*1.0)/df
    avg_IF = (166*0.5 + 165*1.0)/If
    avg_RF = (521*0.5 + 123*1.0)/rf
    avg_DM = (123*0.5 + 606*1.0)/dm
    avg_IM = (166*0.5 + 165*1.0)/im
    avg_RF = (323*0.5 + 323*1.0)/rf
    
    while time < total_time:
        row = random.randint(0,len(edges_list)-1)
        selected_edge = []
        selected_edge.extend(edges_list[row])
    
        column = random.randint(0,1)
        x_i = network.nodes[selected_edge[0]]['opinion']
        x_j = network.nodes[selected_edge[1]]['opinion']
        
        gender0 = network.nodes[selected_edge[0]]['gender']
        gender1 = network.nodes[selected_edge[1]]['gender']
        party0 = network.nodes[selected_edge[0]]['party']
        party1 = network.nodes[selected_edge[0]]['party']
        
        # Concordant Opinions:
        if tau(x_i,x_j) < epsilon(gender0, gender1, party0, party1):
            x_i1 = x_i + mew*(x_j - x_i)
            x_j1 = x_j + mew*(x_i - x_j)
        
        #Discordant Opinions:
        if tau(x_i,x_j) >= epsilon(gender0, gender1, party0, party1):
            x_i1 = x_i - mew*(x_j - x_i - rho(x_j, x_i))
            x_j1 = x_j - mew*(x_i - x_j - rho(x_i, x_j))
        
        #Ensures opinion x remains on range [0, 1]:
        if x_i1 < 0.0:
            x_i1 = 0.0
        elif x_i1 > 1.0:
            x_i1 = 1.0
        if x_j1 < 0.0:
            x_j1 = 0.0
        elif x_j1 > 1.0:
            x_j1 = 1.0
        
        #Checks for zealot status:
        if network.nodes[selected_edge[0]]['zealot'] == "No":
            network.nodes[selected_edge[0]]['opinion'] = x_i1
        if network.nodes[selected_edge[1]]['zealot'] == "No":
            network.nodes[selected_edge[1]]['opinion'] = x_j1
        
        #Adjusts average opinion values for all demographic groups for x_i:
        old0 = x_i
        new0 = network.nodes[selected_edge[0]]['opinion']
        if party0 == "Democrat" and gender0 == "Female":
            avg_DF = avg_DF - (old0 / df) + (new0 / df)
        if party0 == "Independent" and gender0 == "Female":
            avg_IF = avg_IF - (old0 / If) + (new0 / If)
        if party0 == "Republican" and gender0 == "Female":
            avg_RF = avg_RF - (old0 / rf) + (new0 / rf)
        if party0 == "Democrat" and gender0 == "Male":
            avg_DM = avg_DM - (old0 / dm) + (new0 / dm)
        if party0 == "Independent" and gender0 == "Male":
            avg_IM = avg_IM - (old0 / im) + (new0 / im)
        if party0 == "Republican" and gender0 == "Male":
            avg_RM = avg_IM - (old0 / im) + (new0 / im)
        
        #Adjusts opinion proportion for x_i:
        if old0 < 0.33 and new0 >= 0.33:
            counter1 -= 1/4039
        if old0 >= 0.33 and old0 < 0.66:
            if new0 < 0.33 or new0 >= 0.66:
                counter2 -= 1/4039
        if old0 >= 0.66 and new0 < 0.66:
            counter3 -= 1/4039
        
        if new0 < 0.33 and old0 >= 0.33:
            counter1 += 1/4039
        if new0 >= 0.33 and new0 < 0.66:
            if old0 < 0.33 or old0 >= 0.66:
                counter2 += 1/4039
        if new0 >= 0.66 and old0 < 0.66:
            counter3 += 1/4039
        
        #Adjusts average opinion values for all demographic groups for x_j:
        old1 = x_j
        new1 = network.nodes[selected_edge[1]]['opinion']
        if party1 == "Democrat" and gender1 == "Female":
            avg_DF = avg_DF - (old1 / df) + (new1 / df)
        if party1 == "Independent" and gender1 == "Female":
            avg_IF = avg_IF - (old1 / If) + (new1 / If)
        if party1 == "Republican" and gender1 == "Female":
            avg_RF = avg_RF - (old1 / rf) + (new1 / rf)
        if party1 == "Democrat" and gender1 == "Male":
            avg_DM = avg_DM - (old1 / dm) + (new1 / dm)
        if party1 == "Independent" and gender1 == "Male":
            avg_IM = avg_IM - (old1 / im) + (new1 / im)
        if party1 == "Republican" and gender1 == "Male":
            avg_RM = avg_IM - (old1 / im) + (new1 / im)
        
        #Adjusts opinion proportion for x_j:
        if old1 < 0.33 and new1 >= 0.33:
            counter1 -= 1/4039
        if old1 >= 0.33 and old1 < 0.66:
            if new1 < 0.33 or new1 >= 0.66:
                counter2 -= 1/4039
        if old1 >= 0.66 and new1 < 0.66:
            counter3 -= 1/4039
        
        if new1 < 0.33 and old1 >= 0.33:
            counter1 += 1/4039
        if new1 >= 0.33 and new1 < 0.66:
            if old1 < 0.33 or old1 >= 0.66:
                counter2 += 1/4039
        if new1 >= 0.66 and old1 < 0.66:
            counter3 += 1/4039
        
        #Update average opinion storage at each time step:
        DF_l.append(avg_DF)
        IF_l.append(avg_IF)
        RF_l.append(avg_RF)
        DM_l.append(avg_DM)
        IM_l.append(avg_IM)
        RM_l.append(avg_RM)
        
        #Update opinion proportion storage at each time step:
        list1.append(counter1)
        list2.append(counter2)
        list3.append(counter3)
        time_list.append(time)
        
        #Calculate overall average opinion
        nodes_list = list(nx.get_node_attributes(network,'opinion').values()) #the object nx.getnodesattr is a dictionary
        avg_opinion = sum(nodes_list)/len(nodes_list)
        nodes_av.append(avg_opinion)
        nodes.append(nodes_list)
        time += 1      
    
    #Identify winner for this iteration after all time steps are completed:
    winner = 0
    if(counter1 > counter2 and counter1 > counter3):
        winner = 1
    elif(counter2 > counter1 and counter2 > counter3):
        winner = 2
    else:
        winner = 3
        
    return list1, list2, list3, time_list, winner, nodes_av, nodes, DF_l, IF_l, RF_l, DM_l, IM_l, RM_l

In [ ]:
#Execute:
#Variable Inputs:
 #Provides run time
time = 100 
iterations = 1
zealots = 766
mew = 0.45 
#Demographic Inputs:
demographic = [828, 424, 767, 767, 424, 829]
dem_per = [i/4039 for i in demographic]
n = 1000 
DF = [8, 157, 663]
IF = [149, 110, 165]
RF = [123, 521, 123]
DM = [38, 123, 606]
IM = [149, 110, 165]
RM = [299, 439, 91]

print("Erdos Reyni:")
network = nx.erdos_renyi_graph(4039, 0.15)
ER = run(network, time, demographic, DF, IF, RF, DM, IM, RM, zealots, mew)
#Store Final Data:
pickle_out = open("ER.pickle", "wb")
pickle.dump(ER, pickle_out)
pickle_out.close()

print("Facebook:")
df = pd.read_csv("facebook_combined.txt", header = None) #Download facebook_combined.txt
ls = list(df[0])
new_list = [[int(x.split()[0]),int(x.split()[1])] for x in ls]
network = nx.Graph()
network.add_edges_from(new_list)
FB = run(network, time, demographic, DF, IF, RF, DM, IM, RM, zealots, mew)
#Store Final Data:
pickle_out = open("FB.pickle", "wb")
pickle.dump(FB, pickle_out)
pickle_out.close()

Erdos Reyni:
